In [ ]:
# !pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
# !pip3 install torch torchaudio torchvision torchtext torchdata
# !pip install sympy
# !pip install transformers==4.28.0
# !pip install tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# list all directory and files
# %ls "/content/drive/MyDrive/Colab Notebooks/Finance_LLM/"

# Change directory
%cd "/content/drive/MyDrive/Colab Notebooks/Finance_LLM"

/content/drive/MyDrive/Colab Notebooks/Finance_LLM


In [ ]:
from tqdm import tqdm
import json
import os
from datetime import datetime
import time
import logging
from utils import *
from config import parameters as conf
from torch import nn
import torch
import torch.optim as optim


from Model_new import Bert_model

In [ ]:
if conf.pretrained_model == "bert":
    print("Using bert")
    from transformers import BertTokenizer
    from transformers import BertConfig
    tokenizer = BertTokenizer.from_pretrained(conf.model_size)
    model_config = BertConfig.from_pretrained(conf.model_size)

elif conf.pretrained_model == "roberta":
    print("Using roberta")
    from transformers import RobertaTokenizer
    from transformers import RobertaConfig
    tokenizer = RobertaTokenizer.from_pretrained(conf.model_size)
    model_config = RobertaConfig.from_pretrained(conf.model_size)

elif conf.pretrained_model == "finbert":
    print("Using finbert")
    from transformers import BertTokenizer
    from transformers import BertConfig
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model_config = BertConfig.from_pretrained(conf.model_size)

elif conf.pretrained_model == "longformer":
    print("Using longformer")
    from transformers import LongformerTokenizer, LongformerConfig
    tokenizer = LongformerTokenizer.from_pretrained(conf.model_size)
    model_config = LongformerConfig.from_pretrained(conf.model_size)


Using bert


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# create output paths
if conf.mode == "train":
    model_dir_name = conf.model_save_name + "_" + \
        datetime.now().strftime("%Y%m%d%H%M%S")
    model_dir = os.path.join(conf.output_path, model_dir_name)
    results_path = os.path.join(model_dir, "results")
    saved_model_path = os.path.join(model_dir, "saved_model")
    os.makedirs(saved_model_path, exist_ok=False)
    os.makedirs(results_path, exist_ok=False)
    log_file = os.path.join(results_path, 'log.txt')

else:
    saved_model_path = os.path.join(conf.output_path, conf.saved_model_path)
    model_dir_name = datetime.now().strftime("%Y%m%d%H%M%S")
    model_dir = os.path.join(
        conf.output_path, 'inference_only_' + model_dir_name)
    results_path = os.path.join(model_dir, "results")
    os.makedirs(results_path, exist_ok=False)
    log_file = os.path.join(results_path, 'log.txt')


In [ ]:
op_list = read_txt(conf.op_list_file)
op_list = [op + '(' for op in op_list]
op_list = ['EOF', 'UNK', 'GO', ')'] + op_list
const_list = read_txt(conf.const_list_file)
const_list = [const.lower().replace('.', '_') for const in const_list]
reserved_token_size = len(op_list) + len(const_list)

print(op_list)
print(const_list)

Reading: operation_list.txt
Reading: constant_list.txt
['EOF', 'UNK', 'GO', ')', 'add(', 'subtract(', 'multiply(', 'divide(', 'exp(', 'greater(', 'table_sum(', 'table_average(', 'table_max(', 'table_min(']
['const_2', 'const_1', 'const_3', 'const_4', 'const_5', 'const_6', 'const_7', 'const_8', 'const_9', 'const_10', 'const_100', 'const_1000', 'const_10000', 'const_100000', 'const_1000000', 'const_10000000', 'const_1000000000', 'const_m1', '#0', '#1', '#2', '#3', '#4', '#5', '#6', '#7', '#8', '#9', '#10', 'none']


In [ ]:
train_data, train_examples, op_list, const_list = \
    read_examples(input_path=conf.train_file, tokenizer=tokenizer,
                  op_list=op_list, const_list=const_list)

valid_data, valid_examples, op_list, const_list = \
    read_examples(input_path=conf.valid_file, tokenizer=tokenizer,
                  op_list=op_list, const_list=const_list)

test_data, test_examples, op_list, const_list = \
    read_examples(input_path=conf.test_file, tokenizer=tokenizer,
                  op_list=op_list, const_list=const_list)

kwargs = {"examples": train_examples,
          "tokenizer": tokenizer,
          "max_seq_length": conf.max_seq_length,
          "max_program_length": conf.max_program_length,
          "is_training": True,
          "op_list": op_list,
          "op_list_size": len(op_list),
          "const_list": const_list,
          "const_list_size": len(const_list),
          "verbose": True}

train_features = convert_examples_to_features(**kwargs)
kwargs["examples"] = valid_examples
kwargs["is_training"] = False
valid_features = convert_examples_to_features(**kwargs)
kwargs["examples"] = test_examples
test_features = convert_examples_to_features(**kwargs)

Reading /content/drive/MyDrive/Colab Notebooks/Finance_LLM/dataset/train.json


100%|██████████| 6251/6251 [03:07<00:00, 33.39it/s]


Reading /content/drive/MyDrive/Colab Notebooks/Finance_LLM/dataset/dev.json


100%|██████████| 883/883 [00:25<00:00, 34.74it/s]


Reading /content/drive/MyDrive/Colab Notebooks/Finance_LLM/dataset/test.json


100%|██████████| 1147/1147 [00:33<00:00, 34.56it/s]


too long
too long


In [ ]:
def train():
    # keep track of all input parameters
    write_log(log_file, "####################INPUT PARAMETERS###################")
    for attr in conf.__dict__:
        value = conf.__dict__[attr]
        write_log(log_file, attr + " = " + str(value))
    write_log(log_file, "#######################################################")

    model = Bert_model(num_decoder_layers=conf.num_decoder_layers,
                       hidden_size=model_config.hidden_size,
                       dropout_rate=conf.dropout_rate,
                       program_length=conf.max_program_length,
                       input_length=conf.max_seq_length,
                       op_list=op_list,
                       const_list=const_list)

    model = nn.DataParallel(model)
    model.to(conf.device)
    optimizer = optim.Adam(model.parameters(), conf.learning_rate)
    criterion = nn.CrossEntropyLoss(reduction='none', ignore_index=-1)
    model.train()
    # torch.autograd.set_detect_anomaly(True)

    train_iterator = DataLoader(
        is_training=True, data=train_features, batch_size=conf.batch_size, reserved_token_size=reserved_token_size, shuffle=True)

    k = 0
    record_k = 0
    record_loss_k = 0
    loss, start_time = 0.0, time.time()
    record_loss = 0.0

    for _ in range(conf.epoch):
        train_iterator.reset()
        for x in train_iterator:

            input_ids = torch.tensor(x['input_ids']).to(conf.device)
            input_mask = torch.tensor(x['input_mask']).to(conf.device)
            segment_ids = torch.tensor(x['segment_ids']).to(conf.device)
            program_ids = torch.tensor(x['program_ids']).to(conf.device)
            program_mask = torch.tensor(x['program_mask']).to(conf.device)
            option_mask = torch.tensor(x['option_mask']).to(conf.device)

            model.zero_grad()
            optimizer.zero_grad()

            this_logits = model(True, input_ids, input_mask, segment_ids,
                                option_mask, program_ids, program_mask, device=conf.device)

            this_loss = criterion(
                this_logits.view(-1, this_logits.shape[-1]), program_ids.view(-1))
            this_loss = this_loss * program_mask.view(-1)
            # per token loss
            this_loss = this_loss.sum() / program_mask.sum()

            record_loss += this_loss.item()
            record_k += 1
            k += 1

            this_loss.backward()
            optimizer.step()

            if k > 1 and k % conf.report_loss == 0:
                write_log(log_file, "%d : loss = %.3f" %
                          (k, record_loss / record_k))
                record_loss = 0.0
                record_k = 0

            if k > 1 and k % conf.report == 0:
                print("Round: ", k / conf.report)
                model.eval()
                cost_time = time.time() - start_time
                write_log(log_file, "%d : time = %.3f " %
                          (k // conf.report, cost_time))
                start_time = time.time()
                if k // conf.report >= 1:
                    print("Val test")
                    # save model
                    saved_model_path_cnt = os.path.join(
                        saved_model_path, 'loads', str(k // conf.report))
                    os.makedirs(saved_model_path_cnt, exist_ok=True)
                    torch.save(model.state_dict(),
                               saved_model_path_cnt + "/model.pt")

                    results_path_cnt = os.path.join(
                        results_path, 'loads', str(k // conf.report))
                    os.makedirs(results_path_cnt, exist_ok=True)
                    validation_result = evaluate(
                        valid_examples, valid_features, model, results_path_cnt, 'valid')
                    # write_log(log_file, validation_result)

                model.train()

In [ ]:
def evaluate(data_ori, data, model, ksave_dir, mode='valid'):

    pred_list = []
    pred_unk = []

    ksave_dir_mode = os.path.join(ksave_dir, mode)
    os.makedirs(ksave_dir_mode, exist_ok=True)

    data_iterator = DataLoader(
        is_training=False, data=data, batch_size=conf.batch_size_test, reserved_token_size=reserved_token_size, shuffle=False)

    k = 0
    all_results = []
    with torch.no_grad():
        for x in tqdm(data_iterator):

            input_ids = x['input_ids']
            input_mask = x['input_mask']
            segment_ids = x['segment_ids']
            program_ids = x['program_ids']
            program_mask = x['program_mask']
            option_mask = x['option_mask']

            ori_len = len(input_ids)
            for each_item in [input_ids, input_mask, segment_ids, program_ids, program_mask, option_mask]:
                if ori_len < conf.batch_size_test:
                    each_len = len(each_item[0])
                    pad_x = [0] * each_len
                    each_item += [pad_x] * (conf.batch_size_test - ori_len)

            input_ids = torch.tensor(input_ids).to(conf.device)
            input_mask = torch.tensor(input_mask).to(conf.device)
            segment_ids = torch.tensor(segment_ids).to(conf.device)
            program_ids = torch.tensor(program_ids).to(conf.device)
            program_mask = torch.tensor(program_mask).to(conf.device)
            option_mask = torch.tensor(option_mask).to(conf.device)

            logits = model(False, input_ids, input_mask,
                           segment_ids, option_mask, program_ids, program_mask, device=conf.device)

            for this_logit, this_id in zip(logits.tolist(), x["unique_id"]):
                all_results.append(
                    RawResult(
                        unique_id=int(this_id),
                        logits=this_logit,
                        loss=None
                    ))

    output_prediction_file = os.path.join(ksave_dir_mode,
                                          "predictions.json")
    output_nbest_file = os.path.join(ksave_dir_mode,
                                     "nbest_predictions.json")
    output_eval_file = os.path.join(ksave_dir_mode, "full_results.json")
    output_error_file = os.path.join(ksave_dir_mode, "full_results_error.json")

    all_predictions, all_nbest = compute_predictions(
        data_ori,
        data,
        all_results,
        n_best_size=conf.n_best_size,
        max_program_length=conf.max_program_length,
        tokenizer=tokenizer,
        op_list=op_list,
        op_list_size=len(op_list),
        const_list=const_list,
        const_list_size=len(const_list))
    write_predictions(all_predictions, output_prediction_file)
    write_predictions(all_nbest, output_nbest_file)

    if mode == "valid":
        original_file = conf.valid_file
    else:
        original_file = conf.test_file

    exe_acc, prog_acc = evaluate_result(
        output_nbest_file, original_file, output_eval_file, output_error_file, program_mode=conf.program_mode)

    prog_res = "exe acc: " + str(exe_acc) + " prog acc: " + str(prog_acc)
    write_log(log_file, prog_res)

    return

In [ ]:
if __name__ == '__main__':
  if conf.mode == "train":
    train()

####################INPUT PARAMETERS###################
__module__ = config
prog_name = generator
root_path = /content/drive/MyDrive/Colab Notebooks/Finance_LLM/
output_path = /content/drive/MyDrive/Colab Notebooks/Finance_LLM/models/
cache_dir = /content/drive/MyDrive/Colab Notebooks/Finance_LLM/cache/
model_save_name = bert-base
train_file = /content/drive/MyDrive/Colab Notebooks/Finance_LLM/dataset/train.json
valid_file = /content/drive/MyDrive/Colab Notebooks/Finance_LLM/dataset/dev.json
test_file = /content/drive/MyDrive/Colab Notebooks/Finance_LLM/dataset/test.json
op_list_file = operation_list.txt
const_list_file = constant_list.txt
pretrained_model = bert
model_size = bert-base-uncased
retrieve_mode = single
program_mode = seq
device = cuda
mode = train
saved_model_path = /content/drive/MyDrive/Colab Notebooks/Finance_LLM/models/roberta-large-gold_20210713020324/saved_model/loads/119/model.pt
build_summary = False
sep_attention = True
layer_norm = True
num_decoder_layers = 1
ma

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


100 : loss = 2.720
200 : loss = 1.499
300 : loss = 1.352
Round:  1.0
1 : time = 659.119 
Val test


...

IP/2012/page_93.pdf-4
['add(', '231', '205', ')', 'add(', '210', '#0', ')', 'add(', '#1', 'const_3', ')', 'divide(', '#2', 'const_2', ')', 'EOF']
['add(', '231', '205', ')', 'add(', '#0', '210', ')', 'add(', '#1', 'const_3', ')', 'divide(', '#2', 'const_2', ')', 'EOF']
324.5
324.5
IP/2012/page_93.pdf-4
All:  883
Correct:  0
Exe acc:  0.40656851642129105
Prog acc:  0.39184597961494905
exe acc: 0.40656851642129105 prog acc: 0.39184597961494905
3700 : loss = 0.329
3800 : loss = 0.391
3900 : loss = 0.418
Round:  13.0
13 : time = 727.319 
Val test


100%|██████████| 56/56 [00:47<00:00,  1.18it/s]


CB/2010/page_200.pdf-3
['add(', '256868', '1230881', ')', 'add(', '638401', '#0', ')', 'add(', '#1', 'const_3', ')', 'divide(', '#2', 'const_2', ')', 'EOF']
['add(', '256868', '1230881', ')', 'add(', '#0', '638401', ')', 'add(', '#1', 'const_3', ')', 'divide(', '#2', 'const_2', ')', 'EOF']
1063076.5
1063076.5
CB/2010/page_200.pdf-3
IP/2005/page_19.pdf-3
['add(', '1923', '2040', ')', 'add(', '1734', '#0', ')', 'add(', '#1', 'const_3', ')', 'divide(', '#2', 'const_2', ')', 'EOF']
['add(', '1923', '2040', ')', 'add(', '#0', '1734', ')', 'add(', '#1', 'const_3', ')', 'divide(', '#2', 'const_2', ')', 'EOF']
2850.0
2850.0
IP/2005/page_19.pdf-3
All:  883
Correct:  0
Exe acc:  0.41676104190260477
Prog acc:  0.3952434881087203
exe acc: 0.41676104190260477 prog acc: 0.3952434881087203
4000 : loss = 0.304
4100 : loss = 0.344
4200 : loss = 0.346
Round:  14.0
14 : time = 731.780 
Val test


100%|██████████| 56/56 [00:47<00:00,  1.19it/s]


SYY/2019/page_9.pdf-2
['subtract(', '62%', '62%', ')', 'EOF']
['subtract(', '62%', '62%', ')', 'divide(', '#0', '62%', ')', 'EOF']
0.0
0.0
SYY/2019/page_9.pdf-2
BDX/2018/page_82.pdf-1
['add(', '136', '110', ')', 'add(', '81', '#0', ')', 'divide(', '#1', 'const_3', ')', 'EOF']
['add(', '136', '110', ')', 'add(', '#0', '81', ')', 'divide(', '#1', 'const_3', ')', 'EOF']
109.0
109.0
BDX/2018/page_82.pdf-1
LKQ/2010/page_84.pdf-4
['add(', '0.9', '0.9', ')', 'add(', '#0', '1.0', ')', 'EOF']
['add(', '1.0', '0.9', ')', 'add(', '#0', '0.9', ')', 'EOF']
2.8
2.8
LKQ/2010/page_84.pdf-4
TFX/2015/page_70.pdf-3
['multiply(', '86.98', '4446967', ')', 'EOF']
['multiply(', '4446967', '86.98', ')', 'EOF']
386797189.66
386797189.66
TFX/2015/page_70.pdf-3
All:  883
Correct:  0
Exe acc:  0.43148357870894677
Prog acc:  0.4099660249150623
exe acc: 0.43148357870894677 prog acc: 0.4099660249150623
4300 : loss = 0.338
4400 : loss = 0.264
